# Flower Type Detection

## Downloading the dataset

In [ ]:
# from google.colab import files
# files.upload()

# ! mkdir -p ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json


# ! kaggle competitions download -c tpu-getting-started
# ! mkdir data/
# ! unzip tpu-getting-started.zip -d ./data/tpu-getting-started



## Importing Libraries

In [86]:
import re
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras

MODELS_DIR = pathlib.Path('./models/')

print("Tensorflow version " + tf.__version__)


Tensorflow version 2.4.1


## Setting up the TPU

In [87]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2023-01-10 13:48:23.791795: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-01-10 13:48:23.791895: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30021}
2023-01-10 13:48:23.795996: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-01-10 13:48:23.796074: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30021}


REPLICAS:  8


## Importing the data

In [88]:
GCS_DS_PATH = './data/tpu-getting-started'

In [89]:
EPOCHS = 100
BATCH_SIZE = 16 * strategy.num_replicas_in_sync


IMAGE_SIZE = [512, 512]
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'
AUTO = tf.data.experimental.AUTOTUNE

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')


2023-01-10 13:48:29.234765: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-10 13:48:29.278563: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-01-10 13:48:29.321042: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


In [99]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])  # explicit size needed for TPU
    return image


def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string),
        # shape [] means single element
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label  # returns a dataset of (image, label) pairs


def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string),
        # shape [] means single element
        "id": tf.io.FixedLenFeature([], tf.string),
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum  # returns a dataset of image(s)


def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  # disable order, increase speed

    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames)
    # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(
        read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset


def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0, 2)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.keras.image.random_zoom(image, 0.1)
    image = tf.image.random_contrast(image, 0.1, 0.2)
    return image, label


def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    # dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()  # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset


def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset


def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset


def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1))
         for filename in filenames]
    return np.sum(n)


NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(
    NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))


training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()


Dataset: 12753 training images, 3712 validation images, 7382 unlabeled test images


In [101]:
for data_batch, labels_batch in training_dataset:
    print("data batch shape:", data_batch.shape)
    
    print("labels batch shape:", labels_batch.shape)
    break
    

data batch shape: (128, 512, 512, 3)
labels batch shape: (128,)


## Architecturing the Deep Neural Network

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# data_augmentation = keras.Sequential(
#     [
#         tf.keras.RandomFlip(
#             "horizontal_and_vertical"),
#         layers..RandomRotation(0.1),
#         layers.RandomZoom(0.1),
#     ]
# )


with strategy.scope():

    inputs = keras.Input(shape=[*IMAGE_SIZE, 3])
#     x = data_augmentation(inputs)
    x = inputs
#     x = layers.Rescaling(1. / 255)(x)

    x = layers.Conv2D(filters=32, kernel_size=5,
                      use_bias=False, padding='same')(x)

    def conv_layer_with_batch_norm(filter_size, kernel_size=3):
        global x
        x = layers.Conv2D(filters=filter_size,
                          kernel_size=kernel_size,
                          use_bias=False,
                          padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        return x

    x = conv_layer_with_batch_norm(32, kernel_size=5)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

    x = conv_layer_with_batch_norm(32, kernel_size=5)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

    x = conv_layer_with_batch_norm(64, kernel_size=3)
    x = conv_layer_with_batch_norm(64, kernel_size=3)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

    residual = x
    x = conv_layer_with_batch_norm(128, kernel_size=3)
    x = conv_layer_with_batch_norm(128, kernel_size=3)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    residual = layers.Conv2D(128, 1, strides=(2, 2), use_bias=False)(residual)
    residual = layers.BatchNormalization()(residual)
    x = layers.Add()([residual, x])

    residual = x
    x = conv_layer_with_batch_norm(256, kernel_size=3)
    x = conv_layer_with_batch_norm(512, kernel_size=3)
    x = conv_layer_with_batch_norm(256, kernel_size=3)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    residual = layers.Conv2D(256, 1, strides=(2, 2), use_bias=False)(residual)
    x = layers.Add()([residual, x])

    x = layers.Dropout(0.25)(x)

    residual = x
    x = conv_layer_with_batch_norm(512, kernel_size=3)
    x = conv_layer_with_batch_norm(512, kernel_size=3)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    residual = layers.Conv2D(512, 1, strides=(2, 2), use_bias=False)(residual)
    residual = layers.BatchNormalization()(residual)
    x = layers.Add()([residual, x])

    x = conv_layer_with_batch_norm(512, kernel_size=3)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(104, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)


print(model.summary())


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['sparse_categorical_accuracy', ])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                     patience=10,
                                     restore_best_weights=True),
        keras.callbacks.ModelCheckpoint(
        filepath=
        os.path.join(MODELS_DIR, 'modern_arch_ERS.keras'),
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(training_dataset,
                    validation_data=validation_dataset,
                    epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    workers=4,
                    callbacks=callbacks)


## Evaluting the model

In [ ]:
# since we are splitting the dataset and iterating separately on images and ids, order matters.
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))
                ).numpy().astype('U')  # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=[
           '%s', '%d'], delimiter=',', header='id,label', comments='')
